In [3]:
CWD = %pwd
CWD = str(CWD)
import sys
import json
from six.moves import cPickle
import os
import statistics
from collections import defaultdict,Iterable

from IPython.core.display import HTML 
from IPython.core.display import Image, display, display_pretty

import numpy as np
from scipy import misc, polyfit, stats
import scipy
import matplotlib.pyplot as plt
import PIL
# from PIL import Image
%matplotlib inline  
import re

COCO_PATH = os.path.join(CWD,'data','coco')
COCO_ANNO_PATH = os.path.join(COCO_PATH, 'annotations')
sys.path.insert(0, os.path.join(CWD, 'coco', 'PythonAPI'))
from pycocotools.coco import COCO,mask
coco = COCO(os.path.join(COCO_ANNO_PATH,'instances_train2014.json'))

loading annotations into memory...
Done (t=18.12s)
creating index...
index created!


In [4]:
"""
Load Coco data
"""

# load the data from captions
with open(COCO_ANNO_PATH + '/captions_train2014.json') as f:
    coco_captions = json.load(f)
# print len(coco_captions)
# with open(COCO_ANNO_PATH + 'captions_val2014.json') as f:
#     coco_captions =  dict(coco_captions.items() + json.load(f).items())
# print len(coco_captions)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = str(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = str(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)
    img_captions[mid]['captions'].append(cap_info['caption'])

In [15]:
from nltk.corpus import stopwords
sw = [x.upper() for x in stopwords.words('english')]

caps = [ c.replace("."," ").replace(","," ").upper() for _,x in img_captions.iteritems() for c in x['captions']]
break_words = " ".join(caps).split(" ")
all_words = list(set([x for x in break_words if x.isalnum() and x not in sw]))
assert(len(all_words) < 22775)
print len(all_words)



22644


In [16]:
word_idx = {}
for idx in range(len(all_words)):
    word_idx[all_words[idx]] = idx
assert(len(word_idx) == len(all_words))

In [19]:
"""
1. Count word_idx -> frequency
2. Count word_idx_1, word_idx_2 -> coexistence frequency
3. Compute conditional probablity: (coexistence frenquency)/(word frequency)
"""
freq   = np.zeros(len(all_words),                   dtype=np.float64)
comatx = np.zeros((len(all_words), len(all_words)), dtype=np.float64)
for _, data in img_captions.iteritems():
    for cap in data['captions']:
        # get all the word ides
        cap = cap.replace("."," ").replace(","," ").upper()
        cap_words_idx = set([word_idx[x] for x in cap.split(" ") if x.isalnum() and x not in sw])
        
        # increment frequency map
        for widx in cap_words_idx:
            freq[widx] += 1
            
        # increment coexistence map
        for widx in cap_words_idx:
            for vidx in cap_words_idx:
                comatx[widx,vidx] += 1
            
prob_mat = comatx / freq

assert(np.allclose(np.diagonal(prob_mat), np.ones(len(all_words))))
assert(len(freq[freq>0]) == len(all_words))

In [32]:
def cond_prob(w1, given = None):
    if given == None: return 1
    w1idx = word_idx[w1.upper()]
    w2idx = word_idx[given.upper()]
    return prob_mat[w1idx, w2idx]

# high_cond_prob = [x for x in all_words for y in all_words if x != y and cond_prob(x,given=y) > 0.8]
# len(high_cond_prob)
print np.mean(prob_mat), np.sum(freq), np.sum(comatx), np.mean(prob_mat)
# print len(prob_mat[prob_mat>0.0])
(len(prob_mat[prob_mat > 0.9]) - len(all_words))*1.0/(len(all_words)**2)

0.000279648455061 2350789.0 14232419.0 0.000279648455061


9.713101611228111e-05

In [43]:
idxes = np.where(prob_mat>0.999)
import random
while True:
    k = random.randint(0, len(idxes[0]))
    x, y = idxes[0][k], idxes[1][k]
    if x == y: continue
    x, y = all_words[x], all_words[y]
    print x, y
    print cond_prob(x, given = y), cond_prob(y, given = x)
    print freq[word_idx[x.upper()]], freq[word_idx[y.upper()]]
    break

# print cond_prob("winnie", given = "pooh")
# print cond_prob("red", given = "rose"), cond_prob("rose", given = "red")
# # all_words[10]
# print freq[word_idx['winnie'.upper()]], freq[word_idx['pooh'.upper()]]
# print freq[word_idx['red'.upper()]],    freq[word_idx['rose'.upper()]]
# print comatx[word_idx['RED'], word_idx['ROSE']], comatx[word_idx['ROSE'], word_idx['RED']]


CONTAINER THERMAL
1.0 0.00178571428571
560.0 1.0


In [86]:
N = len(all_words)
prob_mat_not_sym = (np.ones((N,N)) - np.diag(np.ones(N))) * prob_mat
ones_idx = np.where(prob_mat_not_sym > 0.9)
print len(ones_idx[0])

49804


In [87]:
def wanted(x,y):
    return not (freq[y] / freq[x] > 100 or freq[x] / freq[y] > 100 or freq[y] < 10 or freq[x] < 10)
clean_idx = [(x,y) for x,y in zip(ones_idx[0], ones_idx[1]) if wanted(x,y)]
print len(clean_idx)

84


In [88]:
import random
# idx = random.randint(0, len(clean_idx[0]));
for idx in range(len(clean_idx)):
    x, y = clean_idx[idx]
    print x, y
    print all_words[x], all_words[y], freq[x], freq[y]
    print cond_prob(all_words[x], given=all_words[y]), cond_prob(all_words[y], given=all_words[x])


445 21901
MOUSE MICKEY 1130.0 21.0
0.952380952381 0.0176991150442
468 11983
LIT DIMLY 987.0 154.0
0.961038961039 0.149949341439
1308 10
POOH WINNIE 14.0 11.0
1.0 0.785714285714
1325 6458
TENNIS RACKETS 11224.0 440.0
0.920454545455 0.0360833927299
1325 7613
TENNIS RACKET 11224.0 3102.0
0.930689877498 0.257216678546
1325 19156
TENNIS COURT 11224.0 4320.0
0.938194444444 0.361101211689
1325 21584
TENNIS RACQUET 11224.0 1553.0
0.938184159691 0.129811119031
1764 10330
TATER TOTS 17.0 15.0
1.0 0.882352941176
2033 8830
ROOM LIVING 12730.0 5420.0
0.935793357934 0.398428908091
2057 14496
WIND INDICATOR 237.0 12.0
0.916666666667 0.0464135021097
2229 19921
WEATHER VEIN 144.0 10.0
1.0 0.0694444444444
2229 22336
WEATHER VANE 144.0 37.0
0.945945945946 0.243055555556
3070 15413
PING PONG 28.0 29.0
0.965517241379 1.0
3186 10888
IRON WROUGHT 167.0 26.0
1.0 0.155688622754
4080 17605
IMAGE UNABLE 2781.0 58.0
0.948275862069 0.019777058612
4275 15045
WHEEL FERRIS 242.0 17.0
1.0 0.0702479338843
4986 1019
KRI

In [84]:
def most_prob_words(word, num = 10):
    idxes = np.argsort(prob_mat[:,word_idx[word.upper()]])
    words = [all_words[idx] for idx in idxes[-2:-num-2:-1]]
#     words = [idx for idx in idxes[:-num-1:-1]]
    return words

print most_prob_words('bathroom')

assert 1.0 == prob_mat[k,k], k

[u'TOILET', u'SINK', u'WHITE', u'MIRROR', u'SHOWER', u'SMALL', u'TUB', u'SITTING', u'NEXT', u'WALL']


In [20]:
all_info = json.load(open(os.path.join(COCO_ANNO_PATH, 'image_info_test2014.json')))
category_names = [x['name'] for x in all_info['categories']]
categories = {x['name']:{} for x in all_info['categories']}
for cate, d in categories.iteritems():
    for word in cate.split(" "):
        d.update({ word : most_prob_words(word)})
print category_names

[u'person', u'bicycle', u'car', u'motorcycle', u'airplane', u'bus', u'train', u'truck', u'boat', u'traffic light', u'fire hydrant', u'stop sign', u'parking meter', u'bench', u'bird', u'cat', u'dog', u'horse', u'sheep', u'cow', u'elephant', u'bear', u'zebra', u'giraffe', u'backpack', u'umbrella', u'handbag', u'tie', u'suitcase', u'frisbee', u'skis', u'snowboard', u'sports ball', u'kite', u'baseball bat', u'baseball glove', u'skateboard', u'surfboard', u'tennis racket', u'bottle', u'wine glass', u'cup', u'fork', u'knife', u'spoon', u'bowl', u'banana', u'apple', u'sandwich', u'orange', u'broccoli', u'carrot', u'hot dog', u'pizza', u'donut', u'cake', u'chair', u'couch', u'potted plant', u'bed', u'dining table', u'toilet', u'tv', u'laptop', u'mouse', u'remote', u'keyboard', u'cell phone', u'microwave', u'oven', u'toaster', u'sink', u'refrigerator', u'book', u'clock', u'vase', u'scissors', u'teddy bear', u'hair drier', u'toothbrush']


In [21]:
categories[category_names[random.randint(0, len(category_names))]]

{u'handbag': [u'HANDBAG',
  u'WOMAN',
  u'CAT',
  u'HOLDING',
  u'NEXT',
  u'PHONE',
  u'SITTING',
  u'CELL',
  u'WALKING']}

In [1]:
"""Helper functions to save/load csr matrix"""
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csc_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])


In [90]:
save_sparse_csr(os.path.join(CWD,'stats','cond_prob','prob_mat.npz'),scipy.sparse.csc_matrix(prob_mat))

In [91]:
with open(os.path.join(CWD,'stats','cond_prob','freq.npz'),'w+') as f:
    np.save(f, freq)

In [92]:
save_sparse_csr(os.path.join(CWD,'stats','cond_prob','comatx.npz'),scipy.sparse.csc_matrix(comatx))

In [93]:
with open(os.path.join(CWD,'stats','cond_prob','words_and_rvwords.pkl'),'w+') as f:
    cPickle.dump(all_words, f, protocol=cPickle.HIGHEST_PROTOCOL)


### For experiments, load the numbers and start from here.
### Do not try to run above cells again. Will take forever.

In [13]:
"""Load numbers"""
prob_mat = load_sparse_csr(os.path.join(CWD,'stats','cond_prob','prob_mat.npz')).todense()
freq = np.load(os.path.join(CWD,'stats','cond_prob','freq.npz'))
comatx = load_sparse_csr(os.path.join(CWD,'stats','cond_prob','comatx.npz')).todense()
with open(os.path.join(CWD,'stats','cond_prob','words_and_rvwords.pkl'),'r') as f:
    all_words = cPickle.load(f)
print len(all_words), len(freq)

22644 22644


### Toilet category experiments with median background filter.

In [82]:
#load results
with open(os.path.join(CWD, 'stats','toilet_median_bg', 'toilet_no_sink_median_bg_71.pkl')) as f:
    d = cPickle.load(f)
ablts = [x['ablt'].upper() for _,x in d.iteritems() if isinstance(x,dict)]
origs = [x['orig'].upper() for _,x in d.iteritems() if isinstance(x,dict)]
print len(ablts), len(origs)

1134 1134


In [78]:
def count_cate_high_coexist_words(cate, d):
    """count the number of images in the category that have the 5 words with highest probability
    occuring in the ablated captions.
    [arg] d - captioning results. imgid -> 'score', 'ablt', 'orig' 
          cate - str
    """
    hfwords = most_prob_words(cate, num = 5)
    ablts = [x['ablt'] for _,x in d.iteritems() if isinstance(x,dict)]
    origs = [x['orig'] for _,x in d.iteritems() if isinstance(x,dict)]
    print hfwords
    def _judge(pair):
        orig,ablt = pair
        for w in hfwords:
#             if w in ablt.upper() and w not in orig.upper():
            if w in ablt.upper():
                return True
        return False
    return len(filter(_judge, zip(origs,ablts)))

print count_cate_high_coexist_words('toilet'.upper(),d), len(d)

[u'BATHROOM', u'SINK', u'WHITE', u'SITTING', u'NEXT']
966 1135


In [83]:
def calc_cond(sentences, word, given):
    word, given = word.upper(), given.upper()
    both = [s for s in sentences if word in s and given in s]
    only_given = [s for s in sentences if given in s]
    return 1.0*len(both)/len(only_given)

cate = 'toilet'
for w in most_prob_words(cate, num = 5):
    print w, calc_cond(origs, w,'toilet'), calc_cond(ablts, w,'toilet'),  cond_prob(w, 'toilet')

BATHROOM 0.864440078585 0.884657236126 0.541167988464
SINK 0.483300589391 0.341675734494 0.284354722422
WHITE 0.23673870334 0.588683351469 0.202307137707
SITTING 0.356581532417 0.47007616975 0.131651045422
NEXT 0.194499017682 0.44613710555 0.116798846431


In [77]:
print len(set(ablts+origs))

374
